## HNSW(Hierarchical Navigable Small World)
HNSW是通过图的方式来解决向量搜索问题的算法，由Y.Malkov与D.Yashunin在[论文](https://arxiv.org/pdf/1603.09320.pdf)中首次提出。

这一个Section安排如下
1. 图拥有什么样的性质可以有效的找到最近的K个向量
2. NSW(Navigable Small World)
3. HNSW(Hierarchical Navigable Small World) 

### 1. 图的性质
我们先直观的感受一下使用图的方式来表现向量空间。
<div style="text-align: center">
<img src="./resources/raw_vector.png"/>
</div>

图中的点代表向量，我们可以看到，如果两个向量的距离较近，那么在图中这两个点之间的距离也会更近。当我们想要通过图的方式来解决向量搜索时，我们会希望从任一点出发可以到达图中其他所有的点，即这个图是一张联通图。
<div style="text-align: center">
<img src="./resources/compare_vec.png"/>
</div>

但仅仅只是联通图，仍然无法做到快速有效的找到距离最近的K个向量。考虑如下的情况，A点与B点之间相隔较远，因此如果想要从A点到达B点需要途经许多点(代表着大量的计算)，同时我们可以看到点C与许多其他的点都有连接，因此如果我们从点C开始寻找距离查询向量最近的K个点，我们会计算大量无关的点(因为与点C相连的点，其中很多大概率是与结果无关的)。
<div style="text-align: center">
<img src="./resources/two_conn.png"/>
</div>

综上所述，为了可以高效而准确的找到距离查询向量最近的K个向量。我们希望构建的图有以下几个性质
1. 联通图(没有孤岛)
2. 距离较远的点,有边可以相连(long range edge)
3. 构建的图中边的数量不宜过多(大量的计算)
4. 距离相近的点，有边连接（保证召回率）
其中3,4是召回率与计算量的tradeoff。





### 2. NSW
NSW通过有效且简单的算法构建出满足上述要求的图，下面分别从构建以及查询两个方面来介绍NSW算法。

#### 1. 构建
首先我们将通过随机的方式，将向量一个一个添加到图中，每一个新添加的点都会与当前图中距离该点最近的**M**个点相连。之所以通过**M**对相连的点的数量进行限制，是为了防止连接的边过多，从而影响查询效率。

我们通过一个例子来描述构建的过程，假设我们将**M**设置为3，并且已经将待加入的向量随机打乱。

首先，添加点A，因为当前图中没有其他任何的点，所以我们只需要添加A，而不用作任何其他的操作。后面我们继续添加点B，此时图中只有点A，点的个数小于3,因此我们可以直接将两者相连。

<div style="text-align: center">
<img src="./resources/addB.png"/>
</div>

类似的我们向图中加入点C，点D，我们会获得以下的图


<div style="text-align: center">
<img src="./resources/addCD.png"/>
</div>

随后我们继续添加点E，此时我们会找到当前图中距离点E最近的**M**个点，即A，B，C并将其相互连接。

<div style="text-align: center">
<img src="./resources/addE.png"/>
</div>

用相同的方式，我们继续添加点F，G，H，最终得到的图如下所示。

<div style="text-align: center">
<img src="./resources/addFH.png"/>
</div>


In [ ]:
import numpy as np
class HNSW:

    def __init__(self, dim, M, efConstruction) -> None:
        """
        Initialize the HNSW object.

        Args:
            dim (int): Dimension of the data.
            M (int): Number of edges per node.
            efConstruction (int): Parameter for controlling the maximum size of the dynamic list.

        Returns:
            None
        """
        self.datas = {}
        self.enterPoint = -1
        self.maxLevel = -1
        self.dim = dim
        self.efConstruction = efConstruction
        self.edges = {}
        self.Mmax = M
        self.Mmax0 = 2 * M
        self.levelMult = 1 / np.log(1.0 * M)
        self.queryIndexes = 100000

    def add(self, point: np.ndarray, label: int) -> None:
        """
        add() adds a point to the HNSW graph.

        Args:
            point (np.array): The point to be added to the graph.
            label (str): The label associated with the point.

        Returns:
            None
        """
        pass 

    def search(self, query: np.ndarray, efSearch: int) -> list:
        """
        search() searches the HNSW graph for the nearest neighbours of a query point.

        Args:
            query (np.array): The query point.
            efSearch (int): The efSearch parameter.

        Returns:
            list: The list of nearest neighbours.
        """
        pass
    def __distance(self, label1: int, label2: int) -> float:
        """
        Calculates the Euclidean distance between two points.

        Args:
            label1 (int): The first point.
            label2 (int): The second point.

        Returns:
            float: The Euclidean distance between x and y.
        """
        return np.linalg.norm(self.datas[label1] - self.datas[label2])

    def __generateLevel(self) -> int:
        """
        Generates a random level for a new node.

        Returns:
            int: The random level.
        """
        distribution = np.random.uniform(0.0, 1.0)
        r = -np.log(distribution) * self.levelMult
        return int(r)

    def __search_layer(self, label: int, eps: set, layer: int, efSearch: int) -> PriorityQueue:
        """
        Searches a single layer of the graph.

        Args:
            label (int): The label of the node to search.
            ep    (int): The label of the enter points.
            layer (int): The layer to search.
            efSearch (int): The efSearch parameter.

        Returns:
            PriorityQueue: The set of currently found nearest neighbours.
        """
        pass 

    def __select_neighbors(self, label: int, candidates: PriorityQueue, M: int) -> set:
        """
        Selects the M nearest neighbours of a node.

        Args:
            label (int): The label of the node.
            candidates (list): The list of candidates.
            M (int): The number of neighbours to select.

        Retruns:
            result(set): The set of M nearest neighbours.
        """
        pass